In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyarrow.parquet as pq

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.parquet', 'sample_submission.csv', 'metadata_train.csv', 'train.parquet', 'metadata_test.csv']


In [2]:
train_data = pq.read_pandas('../input/train.parquet').to_pandas()
train_metadata = pd.read_csv('../input/metadata_train.csv')

In [3]:
train = train_data[:100]
target = train_metadata.target[:100]

In [4]:
#import matplotlib.pyplot as plt
#train.corr().style.format("{:.2}").background_gradient(cmap='PuBu', axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 8712 entries, 0 to 8711
dtypes: int8(8712)
memory usage: 850.9 KB


In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.5, whiten=True)

X_pca = pca.fit_transform(train)

print('Número original de atributos:', train.shape[1])
print('Número reduzido de atributos:', X_pca.shape[1])

Número original de atributos: 8712
Número reduzido de atributos: 10


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Acurácia nos dados originais:', accuracy_score(y_test, y_pred))

#######

X_train, X_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.33, random_state=42)

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Acurácia nos dados reduzidos:', accuracy_score(y_test, y_pred))

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

pca = PCA(n_components=0.5, whiten=True)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Acurácia nos dados originais:', accuracy_score(y_test, y_pred))

Acurácia nos dados originais: 0.9696969696969697
Acurácia nos dados reduzidos: 0.9696969696969697
Acurácia nos dados originais: 0.9696969696969697


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

#sc = StandardScaler()
#X_std = sc.fit_transform(X_train)

fvalue_selector = SelectKBest(f_classif, k=20)
X_kbest = fvalue_selector.fit_transform(X_train, y_train)

print('Número original de atributos:', train.shape[1])
print('Número reduzido de atributos:', X_kbest.shape[1])

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Acurácia nos dados originais:', accuracy_score(y_test, y_pred))

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
model.fit(X_kbest, y_train)
X_test_kbest = fvalue_selector.transform(X_test)
y_pred = model.predict(X_test_kbest)
print('Acurácia nos dados Kbest:', accuracy_score(y_test, y_pred))

Número original de atributos: 8712
Número reduzido de atributos: 20
Acurácia nos dados originais: 0.9696969696969697
Acurácia nos dados Kbest: 1.0


In [8]:
submission = pd.DataFrame()
submission['target'] = y_pred
submission.to_csv('submission.csv', index=False)
submission.head()

,target
0,0
1,0
2,0
3,0
4,0
